# Filtrado Colaborativo: *K-Nearest Neighbors*

Los algoritmos de **filtrado colaborativo** basados en la técnica de los *k* vecinos (KNN) tratan de imitar el comportamiento de los seres humanos cuando buscan recibir una recomendación: cuando necesitamos conocer si nos va a interesar un item, preguntamos a personas que sabemos que conocen nuestros intereses si ellos consideran que el item nos va a gustar.

El método realizará esta misma operación empleando la matriz de votaciones. Este proceso seguirá el siguiente algoritmo:

1. Determinar la similaridad entre los usuarios
2. Encontrar el conjunto de *k* usuarios más similares (*k* vecinos)
3. Estimar las predicciones a los items no votados utilizando las votaciones realizadas por los *k* vecinos
4. (Opcional) Recomendar los *N* items con una predicción más alta

En los siguientes subapartados explicaremos en detalle cada uno de estos pasos.

## Carga del dataset

Para ilustrar mejor el funcionamiento de la técnica de KNN, vamos a desarrollar una implementación explicativa de cómo funciona. 

Para ello usaremos el dataset de [MovieLens 100K](https://grouplens.org/datasets/movielens/) que contiene 100.000 votos de 943 usuarios sobre 1682 películas. Este dataset ha sido dividido en votaciones de entrenamiento (80%) y votaciones de test (20%). Además, los códigos de usuarios e items, han sido modificados para que comience en 0 y terminen en el número de (usuarios / items) - 1.

Inicialmente definimos algunas constantes que nos serán necesarias durante la codificación del algoritmo:

In [0]:
import urllib
import math

In [0]:
NUM_USERS = 943
NUM_ITEMS = 1682

MIN_RATING = 1
MAX_RATING = 5

Cargamos el dataset en la matriz de votaciones. La ausencia de voto se representa con None:

In [0]:
ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

training_file = urllib.request.urlopen("https://drive.google.com/uc?export=view&id=1S4-sxOEvA3MDivaGf7iFirWqt1H6VtaH")
for line in training_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  ratings[int(u)][int(i)] = int(rating)

Del mismo modo, cargamos la matriz de votaciones de test:

In [0]:
test_ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

test_file = urllib.request.urlopen("https://drive.google.com/uc?export=view&id=1LBgTF57DD2NA-petq_FaC1V-h7nrYIh9")
for line in test_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  test_ratings[int(u)][int(i)] = int(rating)

Definimos también algunas funciones auxiliares que nos serán útiles:

In [0]:
def rating_average (u):
  acc = 0
  count = 0
    
  for i in range(NUM_ITEMS):
    if ratings[u][i] != None:
      acc += ratings[u][i]
      count += 1
    
  avg = acc / count
    
  return avg

## Calculo de la similaridad

El primer paso del algoritmo de KNN consiste en determinar el parecido de cada pareja de usuarios basándonos en las votaciones previas de dichos usuarios. Para calcular esta similaridad debemos tener presente que la matriz de votaciones es dispersa y, por lo tanto, parar comparar dos usuarios únicamente podremos emplear los votos sobre aquellos items que sean comunes a ambos usuarios.

Por ejemplo, si un usuario *u* ha realizado las siguientes votaciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|  1 	|  2 	|    	|  4 	|  2 	|    	|  3 	|    	|   	|   5  	|

Y un usuario *v* ha realizado las siguientes votaciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| v 	|  3 	|   	|  2  	|   	|  5 	|  4  	|  3 	|  3  	|   	|  1 	|

Únicamente podremos comparar los votos producidos en los items *i1*, *i5*, *i7* e *i10*.

Existen infinidad de métricas de similaridad que permiten conocer el parecido de dos usuarios en función de sus votos comunes. Las más tradicionales se basan en medidas estadísticas clásicas. Por ejemplo, una de las métricas más empleadas es la **correlación**:

$$sim(u, v) = \frac { 
    \sum_{i \in I_{u,v}} (r_{u,i} - \bar{r}_u)(r_{v,i} - \bar{r}_v) 
   } { 
   \sqrt{ \sum_{i \in I_{u,v}} (r_{u,i} - \bar{r}_u)^2 \sum_{i \in I_{u,v}} (r_{v,i} - \bar{r}_v)^2 } 
   }$$
   
Donde $I_{u,v}$ representa los items que han votado tanto el usuario $u$ como el usuario $v$, $\bar{r}_u$ representa la votación media del usuario $u$ y $\bar{r}_v$ representa la votación media del usuario $v$.

La implementación de esta función podríamos hacerla del siguiente modo:

In [0]:
def correlation_similarity (u, v):
  num = 0
  
  den_u = 0
  den_v = 0
  
  count = 0
  
  avg_u = rating_average(u)
  avg_v = rating_average(v)
  
  for i in range(NUM_ITEMS):
    if ratings[u][i] != None and ratings[v][i] != None:
      r_u = ratings[u][i]
      r_v = ratings[v][i]
      
      num += (r_u - avg_u) * (r_v - avg_v)
      den_u += (r_u - avg_u) * (r_u - avg_u)
      den_v += (r_v - avg_v) * (r_v - avg_v)
      
      count += 1
        
  if count > 0 and den_u != 0 and den_v != 0:    
    cor = num / math.sqrt( den_u * den_v )
    return cor;
  else:
    return None

Aunque las métricas de similaridad basadas en medidas estadísticas clásicas ofrecen valores "aceptables" para el método de los *k* vecinos, existen otras métricas de similaridad específicas para el dominio del filtrado colaborativo. Una de las más populares es **JMSD** que permite calcular la similaridad atendiendo a dos factores: la información no numérica de los votos y la información numérica de los mismos. *JMSD* se define como el producto del índice de Jaccard por la diferencia cuadrática media:

$$JMSD(u,v) = Jaccard(u,v) * (1 - MSD(u, v))$$

El *índice de Jaccard* es una medida que permite comparar la similitud de dos conjuntos. En este caso se emplea para comprobar si los items votados por ambos usuarios coinciden, independientemente de la votación:

$$Jaccard(u,v) =\frac {I_u \cap I_v} {I_u \cup I_v} = \frac {\# \{ i \in I | r_{u,i} \neq \bullet \wedge r_{v,i} \neq \bullet \}} {\# \{ i \in I | r_{u,i} \neq \bullet \vee r_{v,i} \neq \bullet \}}$$

Donde $I_u$ representa los items votados por el usuario $u$ e $I_v$ representa los items votados por el usuario $v$.

El *MSD* permite comparar si los usuarios tienen la misma opinión sobre los items que votaron en común. En este caso, el *MSD* retorna el valor 0 cuando los usuarios votaron idéntico, por lo que suele invertirse para lograr que cuanto más se parezcan dos usuarios, mayor sea su similaridad. Para esto, es necesario normalizar previamente las votaciones:

$$MSD(u,v) = \frac {1} {\#I_{u,v}} \sum_{i \in I_{u,v}} (r_{u,i} - r_{v,i})^2$$

Donde $I_{u,v}$ representa los items votados en común por $u$ y $v$.

Esta medida busca un equilibrio entre los items que los dos usuarios han votado y lo parecidas que son estas votaciones.

La implementación de esta función podríamos hacerla del siguiente modo:

In [0]:
def jmsd_similarity (u, v):
  
  union = 0
  intersection = 0
  diff = 0
  
  for i in range(NUM_ITEMS):
    if ratings[u][i] != None and ratings[v][i] != None:
      r_u = (ratings[u][i] - MIN_RATING) / (MAX_RATING - MIN_RATING)
      r_v = (ratings[v][i] - MIN_RATING) / (MAX_RATING - MIN_RATING)
      
      diff = (r_u - r_v) * (r_u - r_v)
      
      intersection += 1
      union += 1 
      
    elif ratings[u][i] != None or ratings[v][i] != None:  
      union += 1

        
  if intersection > 0:
    jaccard = intersection / union
    msd = diff / intersection
    return jaccard * (1 - msd);
  else:
    return None

## Búsqueda de los *k* vecinos

En el método de KNN, las predicciones se realizan a partir del conjunto de *k* vecinos de cada usuario, esto es, el conjunto de *k* usuarios más similares a uno dado. Este *k* será considerado como un parámetro del sistema y deberá tunearse para cada dataset. 

La búsqueda de los *k* vecinos consiste únicamente en ordenar los usuarios en base a su similaridad y elegir a los *k* con una similaridad más alta.

La implementación de esta función podríamos hacerla del siguiente modo:

In [0]:
k = 25

In [0]:
def get_neighbors (u, similarities):
  
  neighbors = [None for _ in range(k)]
  
  for n in range(k):
    
    max_similarity = 0
    neighbor = None
    
    for v, sim in enumerate(similarities):
      if v not in neighbors and sim != None and sim > max_similarity:
        max_similarity = sim
        neighbor = v
    
    neighbors[n] = neighbor

    return neighbors

## Estimación de las predicciones

La estimación de las predicciones se realiza agregando las votaciones que los *k* vecinos del usuario activo realizaron al item que se quiere predecir. De nuevo, nos encontramos con el problema de la dispersión de la matriz de votaciones, ya que no todos los *k* vecinos habrá votado los mismos items. Podría incluso darse la circunstancia de que un item no hubiera sido votado por ninguno de los *k* vecinos y, por tanto, no podría estimarse una predicción. 

Esta situación puede ayudarnos a tunear el parámetro *k*. Si elegimos un *k* muy bajo conseguimos predicciones precisas ya que se realizar con usuarios muy similares al activo, pero existirán muchos items que no podremos predecir. Si elegimos un *k* muy elevado podremos predecir casi todos los items pero estas predicciones serán poco personalizadas. Llevado al extremo si *k* es igual al número de usuario del sistema (menos uno), las predicciones serían equivalentes a la media de los votos.

Existen diversas formas de agregar las votaciones de los *k* vecinos (*aggregation approach*), aunque, la más popular debido a su sencillez es la media:

$$\hat{r}_{u,i} = \frac{1}{\#N_{u,i}} \sum_{n \in N_{u,i}} r_{n,i}$$

Donde $N_{u,i}$ representa el conjunto de *k* vecinos del usuario $u$ que votaron el item $i$.

La implementación de esta medida de agregación podríamos hacerla del siguiente modo:

In [0]:
def average_prediction (u, i, neighbors):
  acc = 0
  count = 0
  
  for n in neighbors:
    if n == None: break
      
    if ratings[n][i] != None:
      acc += ratings[n][i]
      count += 1
  
  if count > 0:
    prediction = acc / count
    return prediction
  else:
    return None
  

Una evolución simple de esta medida de agregación es la media ponderada, en la cual el voto de cada uno de los vecinos se pondera en función de su similaridad con el usuario sobre el que se está calculando la predicción:

$$\hat{r}_{u,i} = \frac{\sum_{n \in N_{u,i}} sim(u,n) \cdot r_{n,i}}{\sum_{n \in N_{u,i}} sim(u,n)} $$

Donde $N_{u,i}$ representa el conjunto de *k* vecinos del usuario $u$ que votaron el item $i$ y $sim(u,n)$ simboliza la similaridad entre el usuario $u$ y el vecino $n$.

In [0]:
def weighted_average_prediction (u, i, neighbors, similarities):
  num = 0
  den = 0
  
  for n in neighbors:
    if n == None: break
      
    if ratings[n][i] != None:
      num += similarities[n] * ratings[n][i]
      den += similarities[n]
  
  if den > 0:
    prediction = num / den
    return prediction
  else:
    return None

Utilizar la media o la media ponderada como medida de agregación tiene un problema, presupone que todos los usuarios tienen la misma percepción de la escala de votaciones prefijada. Sin embargo, sabemos que esto no es cierto. Existes determinados sesgos que hacen que los usuarios realicen votaciones haciendo una interpretación particular del sentido de su voto. Por ejemplo, existen usuarios más "generosos" con las votaciones que tienden a asignar siempre valoraciones altas y existen usuarios más "tacaños" con las votaciones que tienden a asignar siempre valoraciones más bajas. Que el primer usuario valore un item con 5 y el segundo usuario valore el mismo item con un 4 no quiere decir que al primero le haya gustado más el item. Cada usuario hace su propia interpretación de lo que significan los votos 4 y 5.

Para incluir este fenómeno dentro de las medidas de agregación, es frecuente agregar las votaciones de los k vecinos mediante la **desviación respecto a la media** (*deviation from mean*):

$$\hat{r}_{u,i} = \bar{r}_{u} + \frac{ \sum_{n \in N_{u,i}} r_{n,i} - \bar{r}_n }{\#N_{u,i}}$$

Donde $N_{u,i}$ representa el conjunto de *k* vecinos del usuario $u$ que votaron el item $i$, $\bar{r}_u$ representa la media de votos del usuario $u$ y $\bar{r}_n$ representa la media de votos del usuario $n$.

La implementación de esta medida de agregación podríamos hacerla del siguiente modo:

In [0]:
def deviation_from_mean_prediction (u, i, neighbors):
  acc = 0
  count = 0
  
  for n in neighbors:
    if n == None: break
      
    if ratings[n][i] != None:
      avg_n = rating_average(n)
      acc += ratings[n][i] - avg_n
      count += 1
  
  if count > 0:
    avg_u = rating_average(u)
    prediction = avg_u + acc / count
    return prediction
  else:
    return None

## Cálculo de las recomendaciones

El cálculo de las recomendaciones, por lo general, simplemente implica seleccionar los *N* items con una predicción más alta. Por ejemplo, si quisiéramos recomendar *N = 3* items a un usuario que tuviera las siguientes predicciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|   	|  2,9 	|    	|  4,7 	|  5,0 	|    	|  1,2 	|    	|   	|  3,1 	|

Se le recomendarían a dicho usuario los items *i5*, *i4* e *i10*.

En algunas ocasiones, es posible establecer filtros para acotar los items a recomendar. Por ejemplo: en un sistema de recomendación de restaurantes, es posible filtrar aquellos items que se encuentren a demasiada distancia del usuarios; en un sistema de recomendación de libros, el usuario puede filtrar el idioma o el género literario del libro; en una web de comercio electrónico es posible realizar recomendaciones sobre una categoría concreta...


##Ejemplo de ejecución: cálculo del MAE

En esta sección vamos a mostrar el ejemplo completo de cómo calcular el error medio absoluto (MAE) de las predicciones realizadas por el método de los *k*-vecinos.

Para ello, lo primero que debemos hacer es calcular las predicciones para todos los items que haya recibido una votación de test:

In [0]:
def has_test_ratings (u):
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None:
      return True
  return False

In [0]:
predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

# Rellenamos la matriz de predicciones
for u in range(NUM_USERS):
  if has_test_ratings(u):
    
    # Calcular similaridades
    similarities = [None for _ in range(NUM_USERS)]
    for v in range(NUM_USERS):
      #sim = None if u == v else correlation_similarity(u, v) 
      sim = None if u == v else jmsd_similarity(u, v) 
      similarities[v] = sim
      
    # Calcular vecinos
    neighbors = get_neighbors(u, similarities)
    
    # Calcular predicciones sobre los items de test votados por el usuario
    for i in range(NUM_ITEMS):
      if test_ratings[u][i] != None:
        predictions[u][i] = average_prediction(u, i, neighbors)

Y, a continuación, calculamos el MAE:

In [0]:
def get_user_mae (u):
  mae = 0
  count = 0
  
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      mae += abs(test_ratings[u][i] - predictions[u][i])
      count += 1
  
  if count > 0:
    return mae / count
  else:
    return None

In [0]:
def get_mae ():
  mae = 0
  count = 0
  
  for u in range(NUM_USERS):
    if has_test_ratings(u):
      user_mae = get_user_mae(u)
      
      if user_mae != None:
        mae += user_mae
        count += 1
  
  
  if count > 0:
    return mae / count
  else:
    return None   

In [0]:
mae = get_mae()
print("System MAE = " + str(mae))

System MAE = 0.8773974867724869


## Referencias

Bobadilla, J., Serradilla, F., & Bernal, J. (2010). **A new collaborative filtering metric that improves the behavior of recommender systems**. Knowledge-Based Systems, 23(6), 520-528.

---

*Este documento ha sido desarrollado por **Fernando Ortega**. Dpto. Sistemas Informáticos, ETSI de Sistemas Informáticos, Universidad Politécnica de Madrid.*

*Última actualización: Marzo de 2019*


<img src="https://drive.google.com/uc?export=view&id=1QuQDHyH_yrRbNt6sGzoZ8YcvFGEGlnWZ" alt="CC BY-NC">